# Carwash_세차장
- 다른 프로세스 종료될 때까지 기다리는 시스템
- 세차장 : `Resource`

제한된 수의 `세차 기계`와 랜덤하게 도착하는 여러 대의 `자동차`로 이루어진 세차장 시뮬레이션 환경을 생성합니다.

세차장은 `Resource`를 사용하여 제한된 수의 `세차 기계`를 모델링합니다.

세차장에 차가 도착하면 기계를 요청합니다. 세차 기계를 지급 받으면 세차 프로세스를 시작하고, 세차가 완료될 때까지 기다립니다. 세차가 끝나면 기계를 풀고 차는 떠납니다.

자동차는 설정 프로세스에 의해 생성됩니다. 초기 일정한 수의 차량을 만든 후 시뮬레이션이 계속되는 동안 임의의 시간 간격 후에 새로운 자동차 프로세스를 만들게 됩니다.


In [21]:
import random
import simpy
from functools import partial, wraps

RANDOM_SEED = 42
NUM_MACHINES = 3  # 세차장에 존재하는 정해진 수의 세차 기계
WASHTIME = 3      # 세차하는데 걸리는 시간(분)
T_INTER = 5       # 차량이 생성되는 시간 0-T_INTER(분) 사이에서 랜덤하게 생성
SIM_TIME = 30     # 전체 시뮬레이션 시간(분)

In [17]:
class Carwash(object):
    """세차장은 제한된 수의 세차 기계가 병렬적으로 세차를 수행합니다.

    차가 세차장에 도착하면 세차 기계를 요청하게 되며, 세차 기계를 지급받을 시 세차를 시작합니다.
    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        yield self.env.timeout(WASHTIME)
        self.total += 1
        print("%s 에 대한 세차가 %d%% 진행되었습니다." %
              (car, random.randint(50, 99)))


In [18]:
def car(env, name, cw):
    """name이라는 차가 cw라는 세차장에 도착하면 세차 기계를 요청합니다.
    세차기계를 지급받으면 세차에 돌입하며, 세차가 끝나면 세차장을 떠납니다.
    """
    print('%s이 세차장에 %.2f 분에 도착하였습니다.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s이 %.2f 분에 세차를 시작합니다.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s이 %.2f 분에 세차장을 떠납니다.' % (name, env.now))

In [19]:
def setup(env, num_machines, washtime, t_inter):
    """세차장과 초기 차량 4대, 그리고 시뮬레이션이 계속되는 동안 차량을 생성합니다."""
    
    # 세차장 생성
    carwash = Carwash(env, num_machines, washtime)

    # 4대의 초기 차량 생성
    for i in range(4):
        env.process(car(env, '차량 %d번' % i, carwash))

    # 시뮬레이션이 계속되는 동안 랜덤한 주기로 차량 생성
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, '차량 %d번' % i, carwash))

In [22]:
# Setup and start the simulation
print('######## 세차장 시뮬레이션 시작! #########')
random.seed(RANDOM_SEED)

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

# Execute!
env.run(until=SIM_TIME)


######## 세차장 시뮬레이션 시작! #########
차량 0번이 세차장에 0.00 분에 도착하였습니다.
차량 1번이 세차장에 0.00 분에 도착하였습니다.
차량 2번이 세차장에 0.00 분에 도착하였습니다.
차량 3번이 세차장에 0.00 분에 도착하였습니다.
차량 0번이 0.00 분에 세차를 시작합니다.
차량 1번이 0.00 분에 세차를 시작합니다.
차량 2번이 0.00 분에 세차를 시작합니다.
차량 4번이 세차장에 3.00 분에 도착하였습니다.
차량 0번 에 대한 세차가 97% 진행되었습니다.
차량 1번 에 대한 세차가 67% 진행되었습니다.
차량 2번 에 대한 세차가 65% 진행되었습니다.
차량 0번이 3.00 분에 세차장을 떠납니다.
차량 1번이 3.00 분에 세차장을 떠납니다.
차량 2번이 3.00 분에 세차장을 떠납니다.
차량 3번이 3.00 분에 세차를 시작합니다.
차량 4번이 3.00 분에 세차를 시작합니다.
차량 5번이 세차장에 6.00 분에 도착하였습니다.
차량 3번 에 대한 세차가 58% 진행되었습니다.
차량 4번 에 대한 세차가 97% 진행되었습니다.
차량 5번이 6.00 분에 세차를 시작합니다.
차량 3번이 6.00 분에 세차장을 떠납니다.
차량 4번이 6.00 분에 세차장을 떠납니다.
차량 5번 에 대한 세차가 56% 진행되었습니다.
차량 5번이 9.00 분에 세차장을 떠납니다.
차량 6번이 세차장에 10.00 분에 도착하였습니다.
차량 6번이 10.00 분에 세차를 시작합니다.
차량 6번 에 대한 세차가 55% 진행되었습니다.
차량 6번이 13.00 분에 세차장을 떠납니다.
차량 7번이 세차장에 17.00 분에 도착하였습니다.
차량 7번이 17.00 분에 세차를 시작합니다.
차량 7번 에 대한 세차가 77% 진행되었습니다.
차량 7번이 20.00 분에 세차장을 떠납니다.
차량 8번이 세차장에 24.00 분에 도착하였습니다.
차량 8번이 24.00 분에 세차를 시작합니다.
차량 9번이 세차장에 27.00 분에 도착하였습니다.
차량